# Step 3: LLM Template作成

本ステップでは、質問者から投げかけられたクエリーと類似検索で得られた類似情報を使って、LLMに回答案の作成を依頼するために必要なテンプレートを準備する過程を経験します。
- 各Open LLM向けのテンプレート実装でクラス構造を統一するため、MyTemplateInterfaceインターフェースを用意します
- MyTemplateInterfaceを継承して各Open LLMの仕様に基づき、それぞれのテンプレートをクラスで作成します
- テンプレートにはOpen LLMごとに以下を実装します
    - 類似検索を付与する新規質問
    - 類似検索を使わない新規質問
    - LLMの返答結果から回答を抽出する処理
    - 追加質問
- 実際にOpen LLMをメモリに読み込み、作成したテンプレートの出来具合を試します
![Step3](../image/rag-overview-step3.png)

## 0. 事前準備

### 共通処理/定数定義
バナークラスを読み込みます。

In [ ]:
from mylib.MyBanner import MyBanner

### Access Token入力
Open LLMの取得に必要となるHugging FaceのAccess Tokenを入力します。（事前に発行して入手しておきます）

In [ ]:
MyBanner.start()

from getpass import getpass
HF_ACCESS_TOKEN = getpass("Hugging Face の Access Token を入力して Enter Key を押してください: ")

MyBanner.finish()

### パッケージインストール
本ステップの処理で依存するパッケージをインストールします。

In [ ]:
MyBanner.start()

!python -V
!pip install transformers
!pip install torch
!pip install accelerate
!pip install langchain
!pip install langchain-huggingface

!pip install ipywidgets
!pip install urllib3==1.26.20

!pip install optimum[openvino]

MyBanner.finish()

### import
本ステップの処理で依存するモジュールを読み込みます。

In [ ]:
MyBanner.start()

from transformers import (AutoModelForCausalLM, AutoTokenizer, pipeline)
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
import torch

MyBanner.finish()

## 1. テンプレート作成

### 【定義】MyTemplateInterface
各Open LLM向けのテンプレート実装でクラス構造を統一するためインターフェースを用意します。

In [ ]:
%%writefile mylib/MyTemplateInterface.py
import abc

class MyTemplateInterface(metaclass=abc.ABCMeta):

    test_context = """- 1:
\t- id: 141737
\t- category: 経済産業・社会,対面講座
\t- date: 2025-03-03
\t- title: ビジネスソフト実践科
\t- url: https://www.recurrent-navi.metro.tokyo.lg.jp/course/141737
\t- summary: コースの内容\\nWordやExcelをMOS資格取得レベルまで、またPowerPointの基礎も習得し、業務で活かせるスキルを身に付けます。オリジナルの教材と市販の教材を使用して基礎からしっかりステップアップしていきます。さらに、実務を想定した演習問題で実践力を養います。各種試験会場になっているので、慣れた環境で効率よく受験が可能です。就職支援ではキャリアコンサルタントが常駐しているので、就職相談、応募書類の添削や模擬面接も個別で随時対応します。また、採用実績のある企業の説明会等を実施します。訓練初期から就職に向けての行動計画を立て、スキルを習得しつつ就職に向けて早期に行動するよう促します。駅からも学校からも近い託児施設と契約しているため、希望者の方（条件有り）は安心してお子様を預けて学習に専念することができます。\n※本講座は3/3～5/30の期間を通じて行われます。
\t- 場所: 多摩・島しょ部,北多摩エリア,武蔵野市
\t- 主催者: 東京都（実施機関） 専門学校中野スクールオブビジネス
\t- 定員数: 30名
\t- 費用: 無料（別途、教科書代等は本人負担となります。）
\t- 申込期日: 2025年1月17日
- 2:
\t- id: 83980
\t- category: 経済産業・社会,対面講座
\t- date: 2025-03-08
\t- title: WordPressによるWebサイト制作
\t- url: https://www.recurrent-navi.metro.tokyo.lg.jp/course/83980
\t- summary: WordPressの概要、システムの構成、作業の概要、WordPressの設定、テーマ(テンプレート)の作成、プラグインの作成
\t- 場所: 23区,北区,城北エリア
\t- 主催者: 中央・城北職業能力開発センター赤羽校
\t- 定員数: 29名
\t- 費用: 6,500円
\t- 申込期日: 2024年12月10日
- 3:
\t- id: 111975
\t- category: 経済産業・社会,対面講座
\t- date: 2025-03-02
\t- title: ホームページビルダーによるホームページ作成
\t- url: https://www.recurrent-navi.metro.tokyo.lg.jp/course/111975
\t- summary: ホームページの基礎知識、Webサイトとトップページの作成および編集、リンクの設定、画像の作成と編集\n【ホーページビルダー21】
\t- 場所: 多摩・島しょ部,北多摩エリア,府中市
\t- 主催者: 多摩職業能力開発センター府中校
\t- 定員数: 25名
\t- 費用: 6,500円
\t- 申込期日: 2025年1月10日
- 4:
\t- id: 136726
\t- category: 経済産業・社会,オンライン講座,対面講座
\t- date: null
\t- title: 学校向け出前講座
\t- url: https://www.recurrent-navi.metro.tokyo.lg.jp/course/136726
\t- summary: 消費生活相談や商品テスト指導などの経験を積んだ東京都消費者啓発員（コンシューマー・エイド）が、悪質商法被害の実例に基づき、被害防止の方法・対策について、詳しく解説いたします。\n・契約とは何か／成年年齢引き下げに伴う消費者としての心得\n・悪質商法被害防止（マルチ商法・定期購入など）\n・インターネットやSNSのトラブル防止\n・お金の使い方（キャッシュレス、ローンやクレジットの仕組み）\n・糖度の測定（実験講座）
\t- 場所: オンライン,23区,多摩・島しょ部
\t- 主催者: 申込者
\t- 定員数: 原則 10名以上
\t- 費用: 無料
\t- 申込期日: 遅くても希望日の1か月前
"""

    test_question = "パソコンの使い方を学べるセミナーを教えてください"
    
    @abc.abstractmethod
    def get_template_for_use_retriever(self):
        raise NotImplementedError()
        
    @abc.abstractmethod
    def get_template_for_not_retriever(self):
        raise NotImplementedError()

    @abc.abstractmethod
    def extract_answer_from_response(self, response):
        raise NotImplementedError()

    @abc.abstractmethod
    def get_additional_template_for_conversation(self):
        raise NotImplementedError()

    def test_get_context(self):
        context_str = MyTemplateInterface.test_context
        context_str = context_str.replace('{', '{{')
        context_str = context_str.replace('}', '}}')
        return context_str

    def test_get_question(self):
        return MyTemplateInterface.test_question


### 【定義】LLM別のテンプレ実装
テンプレートの実装をしていますが、完成度はまだまだ未熟です。

#### Gemma by Google
- https://ai.google.dev/gemma/docs/formatting?hl=ja

In [ ]:
%%writefile mylib/MyTemplateImpl4Gemma.py
from mylib.MyTemplateInterface import MyTemplateInterface
import re

class MyTemplateImpl4Gemma(MyTemplateInterface):

    def get_template_for_use_retriever(self):
        template = """<start_of_turn>system
あなたは親切で、礼儀正しく、誠実で優秀な日本人のアシスタントです。
context以下に箇条書きでお伝えする情報を使用してuserからの質問に回答してください。

context:
{context}<end_of_turn>
<start_of_turn>user
{question}<end_of_turn>
<start_of_turn>model"""
        return template
        
    def get_template_for_not_retriever(self):
        template = """<start_of_turn>system
あなたは親切で、礼儀正しく、誠実で優秀な日本人のアシスタントです。
userからの質問に回答してください。<end_of_turn>
<start_of_turn>user
{question}<end_of_turn>
<start_of_turn>model"""
        return template

    def extract_answer_from_response(self, response):
        answer = re.sub(".*<start_of_turn>model", "", response, flags=(re.DOTALL))
        answer = answer.strip()
        return answer

    def get_additional_template_for_conversation(self):
        template = """<end_of_turn>
<start_of_turn>user
{question}<end_of_turn>
<start_of_turn>model
"""
        return template


#### Llama by Meta
- https://www.llama.com/docs/model-cards-and-prompt-formats/meta-llama-2/
- https://www.llama.com/docs/model-cards-and-prompt-formats/meta-llama-3/ 

In [ ]:
%%writefile mylib/MyTemplateImpl4Llama3.py
from mylib.MyTemplateInterface import MyTemplateInterface
import re

class MyTemplateImpl4Llama3(MyTemplateInterface):

    def get_template_for_use_retriever(self):
        template = """<|start_header_id|>system<|end_header_id|>
あなたは親切で、礼儀正しく、誠実で優秀な日本人のアシスタントです。
context以下に箇条書きでお伝えする情報を使用してuserからの質問に回答してください。

context:
{context}<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{question}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>"""
        return template
        
    def get_template_for_not_retriever(self):
        template = """<|start_header_id|>system<|end_header_id|>
あなたは親切で、礼儀正しく、誠実で優秀な日本人のアシスタントです。
userからの質問に回答してください。<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{question}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>"""
        return template

    def extract_answer_from_response(self, response):
        answer = re.sub(".*<|start_header_id|>assistant<|end_header_id|>", "", response, flags=(re.DOTALL))
#        answer = re.sub("\|\|", "", answer, flags=(re.DOTALL))
        answer = answer.strip()
        return answer

    def get_additional_template_for_conversation(self):
        template = """<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{question}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>"""
        return template


#### Phi by Microsoft
- https://huggingface.co/microsoft/Phi-3-mini-128k-instruct#chat-format

In [ ]:
%%writefile mylib/MyTemplateImpl4MsPhi.py
from mylib.MyTemplateInterface import MyTemplateInterface
import re

class MyTemplateImpl4MsPhi(MyTemplateInterface):

    def get_template_for_use_retriever(self):
        template = """<|system|>
あなたは親切で、礼儀正しく、誠実で優秀な日本人のアシスタントです。
context以下に箇条書きでお伝えする情報を使用してuserからの質問に回答してください。

context:
{context}<|end|>
<|user|>
{question}<|end|>
<|assistant|>"""
        return template
        
    def get_template_for_not_retriever(self):
        template = """<|system|>
あなたは親切で、礼儀正しく、誠実で優秀な日本人のアシスタントです。
userからの質問に回答してください。<|end|>
<|user|>
{question}<|end|>
<|assistant|>"""
        return template

    def extract_answer_from_response(self, response):
        answer = re.sub(".*<|assistant|>", "", response, flags=(re.DOTALL))
#        answer = re.sub("^\|\|", "", answer, flags=(re.DOTALL))
        answer = answer.strip()
        return answer

    def get_additional_template_for_conversation(self):
        template = """<|end|>
<|user|>
{question}<|end|>
<|assistant|>"""
        return template


#### OpenCALM by Cyberagent

In [ ]:
%%writefile mylib/MyTemplateImpl4OpenCalm.py
from mylib.MyTemplateInterface import MyTemplateInterface
import re

class MyTemplateImpl4OpenCalm(MyTemplateInterface):

    def get_template_for_use_retriever(self):
        template = """Q: {question}
以下の情報を参考に回答してください。
{context}
A: """
        return template
        
    def get_template_for_not_retriever(self):
        template = """Q: {question}
A: """
        return template

    def extract_answer_from_response(self, response):
        answer = re.sub(".*\nA:", "", response, flags=(re.DOTALL))
        answer = answer.strip()
        return answer

    def get_additional_template_for_conversation(self):
        template = """Q: {question}
A: """
        return template


#### 日本語GPT言語モデル by rinna

In [ ]:
%%writefile mylib/MyTemplateImpl4RinnaJpGpt.py
from mylib.MyTemplateInterface import MyTemplateInterface
import re

class MyTemplateImpl4RinnaJpGpt(MyTemplateInterface):

    def get_template_for_use_retriever(self):
        template = """Q: {question}
以下の情報を参考に回答してください。
{context}
A: """
        return template
        
    def get_template_for_not_retriever(self):
        template = """Q: {question}
A: """
        return template

    def extract_answer_from_response(self, response):
        answer = re.sub(".*\nA:", "", response, flags=(re.DOTALL))
        answer = answer.strip()
        return answer

    def get_additional_template_for_conversation(self):
        template = """Q: {question}
A: """
        return template


#### DeepSeek

In [ ]:
%%writefile mylib/MyTemplateImpl4DeepSeek.py
from mylib.MyTemplateInterface import MyTemplateInterface
import re

class MyTemplateImpl4DeepSeek(MyTemplateInterface):

    def get_template_for_use_retriever(self):
        template = """<｜begin▁of▁sentence｜>
あなたは親切で、礼儀正しく、誠実で優秀な日本人のアシスタントです。
context以下に箇条書きでお伝えする情報を使用してuserからの質問に回答してください。

context:
{context}
<｜User｜>{question}
<｜Assistant｜>"""
        return template
        
    def get_template_for_not_retriever(self):
        template = """<｜begin▁of▁sentence｜>
あなたは親切で、礼儀正しく、誠実で優秀な日本人のアシスタントです。
userからの質問に回答してください。
<｜User｜>{question}
<｜Assistant｜>"""
        return template

    def extract_answer_from_response(self, response):
        answer = re.sub(".*<｜Assistant｜><think>", "", response, flags=(re.DOTALL))
        answer = answer.strip()
        return answer

    def get_additional_template_for_conversation(self):
        template = """<｜User｜>{question}
<｜Assistant｜>"""
        return template


## 2. OpenLLM一覧作成

取り扱うOpen LLMの一覧を配列で列記します。<br>
以下Open LLMの利用には利用規約へ同意が必要のため、事前に利用申請を済ませておきます。
- [Google / Gemma](https://huggingface.co/google/gemma-2-2b-jpn-it)
- [Meta / Llama](https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct)

モデルのダウンロード先は以下になります。
- モデルのダウンロードPATH: ```~/.cache/huggingface/hub/```
- サイズ確認のコマンド: ```$ du -sh ~/.cache/huggingface/hub/*```

### 【定義】MyOpenLlmList Class

In [ ]:
%%writefile mylib/MyOpenLlmList.py

class MyOpenLlmList:

    trained_models = [
        'google/gemma-2b-it',   # 0
        'google/gemma-2-2b-it',   # 1
        'google/gemma-2-2b-jpn-it',   # 2
        'meta-llama/Llama-3.2-1B',  # 3
        'meta-llama/Llama-3.2-1B-Instruct', # 4
        'microsoft/Phi-3-mini-4k-instruct', # 5
        'microsoft/Phi-3-mini-128k-instruct', # 6
        'cyberagent/open-calm-small', # 7
        'cyberagent/open-calm-medium', # 8
        'cyberagent/open-calm-large', # 9
        'rinna/japanese-gpt2-medium', # 10
        'rinna/japanese-gpt-1b', # 11
        'rinna/japanese-gpt-neox-3.6b-instruction-sft-v2', # 12
        'lightblue/DeepSeek-R1-Distill-Qwen-1.5B-Multilingual', #13
    ]

    def __init__(self, enables = []):
        self.__printModelList("Initial list", self.trained_models)
        elements = self.trained_models
        if (len(enables) > 0):
            elements = [self.trained_models[i] for i in enables]
        self.trained_models = elements
        self.__printModelList("Enabled list", self.trained_models)

    def __printModelList(self, caption, models):
        print(f"{caption}:")
        for i, model in enumerate(models):
            print(f" {i}: {model}")

    def getAll(self):
        return self.trained_models

    def get(self, index):
        return self.trained_models[index]


## 3. テンプレート実装確認

### OpenLLM一覧生成

In [ ]:
MyBanner.start()

from mylib.MyOpenLlmList import MyOpenLlmList

openllm_list = MyOpenLlmList()

MyBanner.finish()

### 対象のLLMとテンプレ選定
テンプレートの実装を確認する対象Open LLMを一つ選定します。

In [ ]:
MyBanner.start()

from mylib.MyTemplateImpl4Gemma import MyTemplateImpl4Gemma
from mylib.MyTemplateImpl4MsPhi import MyTemplateImpl4MsPhi
from mylib.MyTemplateImpl4OpenCalm import MyTemplateImpl4OpenCalm
from mylib.MyTemplateImpl4RinnaJpGpt import MyTemplateImpl4RinnaJpGpt
from mylib.MyTemplateImpl4Llama3 import MyTemplateImpl4Llama3
from mylib.MyTemplateImpl4DeepSeek import MyTemplateImpl4DeepSeek

# select a model
model_name = openllm_list.get(2)
print(f"{model_name=}")

# a template will be selected
template_obj = None
if 'google/gemma' in model_name:
    template_obj = MyTemplateImpl4Gemma()
elif 'microsoft/Phi' in model_name:
    template_obj = MyTemplateImpl4MsPhi()
elif 'cyberagent/open-calm' in model_name:
    template_obj = MyTemplateImpl4OpenCalm()
elif 'rinna/japanese-gpt' in model_name:
    template_obj = MyTemplateImpl4RinnaJpGpt()
elif 'meta-llama/Llama-3' in model_name:
    template_obj = MyTemplateImpl4Llama3()
elif '/DeepSeek' in model_name:
    template_obj = MyTemplateImpl4DeepSeek()
print(f"{template_obj=}")
print("="*40)

template_str = template_obj.get_template_for_use_retriever()
#template_str = template_obj.get_template_for_not_retriever()
print(f"{template_str=}")
if '{context}' in template_str:
    context_str = template_obj.test_get_context()
    template_str = template_str.replace('{context}', context_str)
print(f"{template_str=}")

question = template_obj.test_get_question()
print(f"{question=}")

MyBanner.finish()

### テンプレの実装確認
テンプレートの実装を確認します。

In [ ]:
MyBanner.start()

MyBanner.passing("01/10: start: check the environment for Intel hardware or other hardware")
IS_AVAILABLE_OVMODEL = False
if torch.backends.mps.is_available() or torch.cuda.is_available():
    print("- MPS or CUDA is available.")
else:
    try:
        from optimum.intel import OVModelForCausalLM
        IS_AVAILABLE_OVMODEL = True
        print("- OpenVINO (optimum-intel) is available.")
    except ImportError:
        print("- OpenVINO (optimum-intel) not found. Will use PyTorch (AutoModelForCausalLM).")

MyBanner.passing("02/10: start: it will create model.")
if IS_AVAILABLE_OVMODEL:
    print("- It will use OVModelForCausalLM.from_pretrained().")
    model = OVModelForCausalLM.from_pretrained(
        model_name,
        export = True,
        device = "auto",
        token = HF_ACCESS_TOKEN,
    )
    print("- model was created by OVModelForCausalLM.from_pretrained().")
else:
    print("- It will use AutoModelForCausalLM.from_pretrained().")
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map = "auto",
        low_cpu_mem_usage = True,
        torch_dtype = "auto",
        trust_remote_code = True,
        token = HF_ACCESS_TOKEN,
    )
    print("- model was created by AutoModelForCausalLM.from_pretrained().")

print(f"{model.device=}")

MyBanner.passing("03/10: start: tokenizer = AutoTokenizer.from_pretrained()")
tokenizer = AutoTokenizer.from_pretrained(model_name, token = HF_ACCESS_TOKEN)

MyBanner.passing("04/10: start: pipe = pipeline()")
pipe = pipeline(
    'text-generation',
    model = model,
    tokenizer = tokenizer,
    max_new_tokens = 1024,
    torch_dtype = "auto",
)

MyBanner.passing("05/10: start: llm = HuggingFacePipeline()")
llm = HuggingFacePipeline(
    pipeline=pipe
)

MyBanner.passing("06/10: start: template = PromptTemplate.from_template()")
template = PromptTemplate.from_template(template_str)

MyBanner.passing("07/10: start: create chain")
chain = (
    {"question": RunnablePassthrough()}
    | template
    | llm
)
print(f"{chain=}")

MyBanner.passing("08/10: start: chain.invoke(question)")
response = chain.invoke(question)
print(f"{response=}")

MyBanner.passing("09/10: start: extract an answer from response")
answer = template_obj.extract_answer_from_response(response)
print(f"{answer=}")

MyBanner.passing("10/10: start: display the first few characters of the answer in hex")
ascii_list = list(answer[:10])
hex_list=[f'0x{ord(i):02X}' for i in ascii_list]
hex_string=' '.join(hex_list)
print(f'# {hex_string=}')

MyBanner.finish()

## 3. 本ステップを終えて

ここまでの手順でLLMに回答案の生成を依頼するためのテンプレートを作成する過程を経験しました。次のステップではここまでに経験してきた類似検索と準備したテンプレートを活用してRetrieverとGeneratorの構築を経験します。
- 次のStep ≫ [Step 4: Retriever+Generator構築](./rag-step04-retriever_and_generator.ipynb)
- 今のStep ≫ Step 3: LLM Template作成
- 前のStep ≫ [Step 2: Vector DBで類似検索](./rag-step02-search_from_vectordb.ipynb)